In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from tickers import getTickerList, getCandlePatternList
from datemodule import getDateRange
from indicators import ChannelBreakoutIndicator, SupportResistanceIndicator
from datasource import getFullData
import pandas

In [6]:
import talib
import pandas_ta as ta
from tqdm import tqdm

tickers = getTickerList(all=True)
tickerShortlist = {}
tickerShortlistData = {}

for i in tqdm(range(len(tickers))):
    ticker = tickers[i]
    resultDict = {}
    tickerShortlistData[ticker] = {}

    # Fetch Data
    startDate, endDate = getDateRange('1y')
    df = getFullData(ticker, startDate, endDate)
    
    candleIndex = len(df)-1
    
    # RSI
    df['RSI'] = ta.rsi(df.Close, length=14)

    # Candlestick Pattern Recognition
    op = df['Open']
    hi = df['High']
    lo = df['Low']
    cl = df['Close']
    for pattern in getCandlePatternList(all=False):
        sig = getattr(talib, pattern)(op, hi, lo, cl)[candleIndex]
        if sig != 0:
            resultDict[pattern] = sig
    
    # Channel Breakout Indicator Signal
    cbi = ChannelBreakoutIndicator(df, ticker)
    cbi.calculate(40)
    cbSignal = cbi.getBuySell()[-1]
    if (cbSignal != ''):
        resultDict["ChannelBreakoutIndicator"] = cbSignal
        
    # Support Resistance Indicator Signal
    sri = SupportResistanceIndicator(df, 11, 5, ticker)
    sri.calculate()
    srSignal = sri.getBuySell()[-1]
    if (srSignal != ''):
        resultDict["SupportResistanceIndicator"] = srSignal

    # Collect Signals
    if (len(resultDict) != 0):
        resultDict["RSI"] = df['RSI'][candleIndex]
        tickerShortlistData[ticker]["CBI"] = cbi
        tickerShortlistData[ticker]["SRI"] = sri
        tickerShortlist[ticker] = resultDict

100%|██████████| 96/96 [06:12<00:00,  3.88s/it]


In [7]:
shortlisted = pandas.DataFrame(tickerShortlist).transpose()
shortlisted["signalCount"] = [row.count() for index, row in shortlisted.iterrows()]
shortlisted = shortlisted.sort_values(by='signalCount', ascending=False).drop(['signalCount'], axis=1)
shortlisted

,ChannelBreakoutIndicator,RSI,CDLHARAMI,SupportResistanceIndicator,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLENGULFING
HAVELLS,SELL,44.123448,NaN,NaN,NaN,NaN,-100
ZOMATO,NaN,50.727699,NaN,BUY,NaN,NaN,-100
RELIANCE,BUY,66.858998,-100,NaN,NaN,NaN,NaN
UPL,SELL,38.751811,NaN,NaN,NaN,NaN,-100
HINDUNILVR,NaN,53.93317,NaN,NaN,-100.0,-100.0,NaN
LICI,NaN,57.271694,NaN,BUY,NaN,NaN,NaN
HAL,NaN,62.21962,NaN,NaN,NaN,NaN,100.0
BERGEPAINT,NaN,56.359035,NaN,NaN,NaN,NaN,-100.0
PAGEIND,NaN,31.603383,NaN,NaN,NaN,NaN,-100.0
AWL,SELL,40.90107,NaN,NaN,NaN,NaN,NaN


In [8]:
for ticker in shortlisted.index:
    print(tickerShortlist[ticker])
    i = tickerShortlistData[ticker]["CBI"]
    i.showIndicator(len(i.df.index)-1, 40)

{'CDLENGULFING': -100, 'ChannelBreakoutIndicator': 'SELL', 'RSI': 44.123448281078076}
0.24778363458300504 0.9761137034503482


{'CDLENGULFING': -100, 'SupportResistanceIndicator': 'BUY', 'RSI': 50.727698658650425}
0.9402366981790162 0.8408608177269709


{'CDLHARAMI': -100, 'ChannelBreakoutIndicator': 'BUY', 'RSI': 66.85899844556249}
0.9320660390764998 0.9861452597409583


{'CDLENGULFING': -100, 'ChannelBreakoutIndicator': 'SELL', 'RSI': 38.7518105148353}
0.3909399926959395 0.16474464579901313


{'CDLEVENINGDOJISTAR': -100, 'CDLEVENINGSTAR': -100, 'RSI': 53.933170470769795}
0.8824827601850566 0.6068847781563117


...

{'SupportResistanceIndicator': 'BUY', 'RSI': 52.61399983377465}
0.8726697003985296 0.9919843541759746


{'CDLHARAMI': -100, 'RSI': 51.967635409175784}
0.3290011778918873 0.9608803362427328


{'CDLHARAMI': -100, 'RSI': 66.74304596529835}
0.36033390741910926 0.6069371734677128


{'CDLHARAMI': 100, 'RSI': 68.34563819792018}
1.0 1.0


{'CDLHARAMI': -100, 'RSI': 59.27345433200746}
1.0 0.9589633823948842


In [9]:
for ticker in shortlisted.index:
    print(tickerShortlist[ticker])
    i = tickerShortlistData[ticker]["SRI"]
    i.showIndicator(len(i.df.index)-1)

{'CDLENGULFING': -100, 'ChannelBreakoutIndicator': 'SELL', 'RSI': 44.123448281078076}


{'CDLENGULFING': -100, 'SupportResistanceIndicator': 'BUY', 'RSI': 50.727698658650425}


{'CDLHARAMI': -100, 'ChannelBreakoutIndicator': 'BUY', 'RSI': 66.85899844556249}


{'CDLENGULFING': -100, 'ChannelBreakoutIndicator': 'SELL', 'RSI': 38.7518105148353}


{'CDLEVENINGDOJISTAR': -100, 'CDLEVENINGSTAR': -100, 'RSI': 53.933170470769795}


...

{'SupportResistanceIndicator': 'BUY', 'RSI': 52.61399983377465}


{'CDLHARAMI': -100, 'RSI': 51.967635409175784}


{'CDLHARAMI': -100, 'RSI': 66.74304596529835}


{'CDLHARAMI': 100, 'RSI': 68.34563819792018}


{'CDLHARAMI': -100, 'RSI': 59.27345433200746}
